This notebook migrates the SQLite Database to Postgresql to be able to access it remotely for the dashboard

In [1]:
import pandas as pd
def remove_null_bytes(df):
    # removes null bytes which are not allowed in postgresql#
    for col in df.select_dtypes(include=['object']):
        df[col] = df[col].apply(lambda x: x.replace('\x00', '') if isinstance(x, str) else x)
    return df

In [15]:
import sqlite3
import csv

#export SQL tables db to csv
sqlite_conn = sqlite3.connect('book_reviews.db')
tables = sqlite_conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
dataframes = {}
for table_name in tables:
    #filter out sqlite metadata
    if table_name[0] == 'sqlite_sequence':
        continue
    table_name = table_name[0]
    df = pd.read_sql_query(f"SELECT * FROM {table_name};", sqlite_conn)
    dataframes[table_name] = remove_null_bytes(df)
    #df.to_csv(f"{table_name}.csv", index=False,escapechar='\\',quoting=csv.QUOTE_ALL)



In [ ]:
import psycopg2
from dotenv import load_dotenv
import os
def connectToDatabase():
    # Connect to Supabase database, has to be closed afterwards
    load_dotenv()
    USER = os.getenv("USER")
    PASSWORD = os.getenv("PASSWORD")
    HOST = os.getenv("HOST")
    PORT = os.getenv("PORT")
    DBNAME = os.getenv("DBNAME")
    try:
        connection = psycopg2.connect(
            user=USER,
            password=PASSWORD,
            host=HOST,
            port=PORT,
            dbname=DBNAME
        )
        
        print(f"Connection successful! to {HOST} on port {PORT}")
        return connection

    except Exception as e:
        print(f"Failed to connect: {e}")


In [16]:
import re
connection = connectToDatabase()
#read in SQL schema
with open("db_schema.sql", "r") as file:
    schema_sqlite = file.read()
#adjust for PostgreSQL
schema_postgresql = re.sub(r'INTEGER ((\w* )*)AUTOINCREMENT',r'SERIAL \1',schema_sqlite)
# apply schema to  Database
with connection.cursor() as cur:
    cur.execute(schema_postgresql)
    connection.commit()
    connection.close()

Connection successful! to aws-0-eu-central-1.pooler.supabase.com on port 6543


In [ ]:
from psycopg2.extras import execute_values

connection = connectToDatabase()

for table_name, df in dataframes.items():
    columns = ','.join(df.columns)
    values = [tuple(x) for x in df.to_numpy()]
    insert_query = f"INSERT INTO {table_name} ({columns}) VALUES %s"
    with connection.cursor() as cur:
        execute_values(cur, insert_query, values)
        connection.commit()
 
connection.close()


Connection successful! to aws-0-eu-central-1.pooler.supabase.com on port 6543
